In [29]:
import subprocess
import json

from dateutil.parser import parse as parse_datetime
import pandas as pd
import numpy as np

from helpers import *

In [2]:
TARGET_COLUMNS = [
    "hospital_id",
    #'row_id',
    "line_type",
    "description",
    "rev_code",
    "local_code",
    "code",
    "ms_drg",
    "apr_drg",
    "eapg",
    "hcpcs_cpt",
    "modifiers",
    "alt_hcpcs_cpt",
    "thru",
    "apc",
    "icd",
    "ndc",
    "drug_hcpcs_multiplier",
    "drug_quantity",
    "drug_unit_of_measurement",
    "drug_type_of_measurement",
    "billing_class",
    "setting",
    "rate_category",
    "payer_name",
    "plan_name",
    "standard_charge",
    "standard_charge_percent",
    "contracting_method",
    "additional_generic_notes",
    "additional_payer_specific_notes",
]

In [3]:
TRANSPARENCY_PAGE = "https://www.mskcc.org/insurance-assistance/understanding-cost-your-care-msk/download"

urls = ["https://www.mskcc.org/teaser/standard-charges-nyc.json", 
        "https://www.mskcc.org/teaser/standard-charges-ny-suburbs.json",
        "https://www.mskcc.org/teaser/standard-charges-sourthern-nj.json",
        "https://www.mskcc.org/teaser/standard-charges-northern-nj.json"]
url = urls[0]
ccn = "330154"
ein = "13-1624082"
filename = derive_filename_from_url(url)
filename

'standard-charges-nyc.json'

In [4]:
subprocess.run(["wget", "--no-clobber", url, "-O", filename])

File ‘standard-charges-nyc.json’ already there; not retrieving.


CompletedProcess(args=['wget', '--no-clobber', 'https://www.mskcc.org/teaser/standard-charges-nyc.json', '-O', 'standard-charges-nyc.json'], returncode=1)

In [5]:
in_f = open(filename, "r")
json_dict = json.load(in_f)
in_f.close()
json_dict.keys()

dict_keys(['File Notes', 'OP_NYC_07272023', 'IP_NYC_07272023', 'PHY_NYC_07272023'])

In [6]:
json_dict['File Notes']

[{'Memorial Hospital for Cancer and Allied Diseases': 'New York City Outpatient, Inpatient and Physician Charges'},
 {'Memorial Hospital for Cancer and Allied Diseases': 'Updated 07/27/2023'}]

In [31]:
date_str = json_dict['File Notes'][-1]['Memorial Hospital for Cancer and Allied Diseases']
date_str = date_str.split(" ")[-1]
last_updated = parse_datetime(date_str).isoformat().split("T")[0]
last_updated

'2023-07-27'

In [7]:
df_op = pd.DataFrame(json_dict['OP_NYC_07272023'])
df_op

,CHG CD,CHG CD DESC,MOD,MCD CPT,REV CD,PRICE,SELF PAY,AETNA COMMERCIAL NYC RATES,AETNA PREMIER CARE NETWORK NYC RATES,AETNA CONTRACTING METHOD,...,HEALTHFIRST MCAID & ESS 3&4 MR/DD/TBI PTS RATES,HEALTHFIRST MCAID & ESS 3&4 HOSP OP DEPT RATES,FIDELIS ESS 1&2 AMB SURG RATES,FIDELIS ESS 1&2 MR/DD/TBI PTS RATES,FIDELIS ESS 1&2 HOSP OP DEPT RATES,FIDELIS MCAID & ESS 3&4 AMB SURG RATES,FIDELIS MCAID & ESS 3&4 MR/DD/TBI PTS RATES,FIDELIS MCAID & ESS 3&4 HOSP OP DEPT RATES,MIN OP,MAX OP
0,AMB000007,HOSPITAL OUTPT VISIT - AMB,,G0463,510,$393.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AMB000010,INJECTION TENDON SHEATH/LIGMNT,,20550,490,$740.00,NaN,NaN,NaN,NaN,...,355.789637,296.510025,552.987118,622.631864,518.892544,315.992639,355.789637,296.510025,NaN,NaN
2,AMB000012,"N BLOCK INJ, CELIAC PELUS",,64530,490,"$1,604.00",NaN,NaN,NaN,NaN,...,675.262875,562.754481,1049.529366,1181.710032,984.820342,599.731066,675.262875,562.754481,NaN,NaN
3,AMB000016,SSEP STD 1 LWR LIMB - AMB,52,95926,922,$562.00,NaN,438.36,394.524,PERCENT OF CHARGE,...,166.310369,138.600697,258.488394,291.043146,242.551220,147.707654,166.310369,138.600697,NaN,NaN
4,AMB000017,SSEP STD 1 UPR LIMB- AMB,52,95925,922,$562.00,NaN,438.36,394.524,PERCENT OF CHARGE,...,166.310369,138.600697,258.488394,291.043146,242.551220,147.707654,166.310369,138.600697,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19959,SERVICE PACKAGE,NaN,NaN,S2325,NaN,NaN,NaN,NaN,NaN,CASE RATE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19960,SERVICE PACKAGE,NaN,NaN,S2340,NaN,NaN,NaN,NaN,NaN,CASE RATE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19961,SERVICE PACKAGE,NaN,NaN,S2342,NaN,NaN,NaN,NaN,NaN,CASE RATE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19962,SERVICE PACKAGE,NaN,NaN,S2411,NaN,NaN,NaN,NaN,NaN,CASE RATE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_ip = pd.DataFrame(json_dict['IP_NYC_07272023'])
df_ip

,CHG CD,CHG CD DESC,MOD,MCD CPT/MS DRG,REV CD,PRICE,SELF PAY,AETNA COMMERCIAL NYC RATES,AETNA PREMIER CARE NETWORK NYC RATES,AETNA CONTRACTING METHOD,...,URN CONTRACTING METHOD,EMPIRE HEALTHPLUS RATES,EMBLEM HIP ESSENTIAL 1&2 RATES,EMBLEM HIP MCAID & ESSENTIAL 3&4 RATES,HEALTHFIRST ESSENTIAL 1&2 RATES,HEALTHFIRST MCAID & ESSENTIAL 3&4 RATES,FIDELIS ESSENTIAL 1&2 RATES,FIDELIS MCAID & ESSENTIAL 3&4 RATES,MIN IP,MAX IP
0,AMB000007,HOSPITAL OUTPT VISIT - AMB,,G0463,510,$393.00,$393.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AMB000010,INJECTION TENDON SHEATH/LIGMNT,,20550,490,$740.00,$740.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AMB000012,"N BLOCK INJ, CELIAC PELUS",,64530,490,"$1,604.00","$1,604.00",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AMB000016,SSEP STD 1 LWR LIMB - AMB,52,95926,922,$562.00,$562.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AMB000017,SSEP STD 1 UPR LIMB- AMB,52,95925,922,$562.00,$562.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13912,SERVICE PACKAGE,Per Diem: Med/Surg/Ped/OB,NaN,NaN,Other IP,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13913,SERVICE PACKAGE,BONE MARROW TRANSPLANTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13914,SERVICE PACKAGE,CAR-T TRANSPLANTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13915,SERVICE PACKAGE,Per Diem: Surg Day 0-1,NaN,Surg DRGs,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_phy = pd.DataFrame(json_dict['PHY_NYC_07272023'])
df_phy

,CHG CD,CPT Code,Mod,Description,PRICE,SELF PAY,AETNA COMMERCIAL NYC RATES,AETNA PREMIER CARE NETWORK NYC RATES,AETNA CONTRACTING METHOD,CIGNA RATES,...,EMBLEM HIP ESS 1&2 RATES,EMBLEM MCAID & HIP ESS 3&4 RATES,HEALTHFIRST ESS 1&2 RATES,HEALTHFIRST MCAID & ESS 3&4 RATES,HEALTHFIRST MCAID & ESS CONTRACTING METHODS,FIDELIS ESS 1&2 RATES,FIDELIS MCAID & ESS 3&4 RATES,FIDELIS MCAID & ESS CONTRACTING METHODS,MIN PHY,MAX PHY
0,0100100,00100,,ANESTHESIA FOR SALIVARY GLANDS,145,145,119.3300,113.36350,Conv Factor,134.0000,...,MEDICAID RATE,MEDICAID RATE,NaN,NaN,FEE SCHEDULE,NaN,NaN,FEE SCHEDULE,NaN,NaN
1,0100102,00102,,PLASTIC REPAIR OF CLEFT LIP,145,145,119.3300,113.36350,Conv Factor,134.0000,...,MEDICAID RATE,MEDICAID RATE,NaN,NaN,FEE SCHEDULE,NaN,NaN,FEE SCHEDULE,NaN,NaN
2,0100103,00103,,BLEPHAROPLASTY,145,145,119.3300,113.36350,Conv Factor,134.0000,...,MEDICAID RATE,MEDICAID RATE,NaN,NaN,FEE SCHEDULE,NaN,NaN,FEE SCHEDULE,NaN,NaN
3,0100104,00104,,ANESTH FOR ELECTROCONVULSIVE THERAPY,145,145,119.3300,113.36350,Conv Factor,134.0000,...,MEDICAID RATE,MEDICAID RATE,NaN,NaN,FEE SCHEDULE,NaN,NaN,FEE SCHEDULE,NaN,NaN
4,0100120,00120,,ANESTHESIA FOR EAR SURGERY,145,145,119.3300,113.36350,Conv Factor,134.0000,...,MEDICAID RATE,MEDICAID RATE,NaN,NaN,FEE SCHEDULE,NaN,NaN,FEE SCHEDULE,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12987,9300452,G0452,26,MOLECULAR PATH PROC; INTRP AND RPT- TIER 1,205,205,75.7914,72.00183,FEE SCHEDULE/PERCENT OF CHARGE,31.4959,...,MEDICAID RATE,MEDICAID RATE,55.77,44.616,FEE SCHEDULE,55.77,44.616,FEE SCHEDULE,NaN,NaN
12988,9310452,G0452,26,MOLECULAR PATH PROC; INTRP AND RPT- TIER 2,205,205,75.7914,72.00183,FEE SCHEDULE/PERCENT OF CHARGE,31.4959,...,MEDICAID RATE,MEDICAID RATE,55.77,44.616,FEE SCHEDULE,55.77,44.616,FEE SCHEDULE,NaN,NaN
12989,9388387,88387,26,"MACROSCOPIC EXAM PREP,MOLEC STUDIES, EA TISS",150,150,111.6870,106.10265,FEE SCHEDULE/PERCENT OF CHARGE,183.2454,...,MEDICAID RATE,MEDICAID RATE,38.39,30.712,FEE SCHEDULE,38.39,30.712,FEE SCHEDULE,NaN,NaN
12990,J730842,J7308,,"AMINOLEVULINIC ACID HCL ADM,20%, 354MG - HAUP ...",1135,1135,567.5000,539.12500,FEE SCHEDULE/PERCENT OF CHARGE,933.0350,...,MEDICAID RATE,MEDICAID RATE,NaN,NaN,FEE SCHEDULE,NaN,NaN,FEE SCHEDULE,NaN,NaN


In [10]:
def rearrange_columns(df):
    columns = df.columns.to_list()

    new_columns = list(filter(lambda c: "CONTRACTING METHOD" in c, columns))
    new_columns += list(filter(lambda c: not "CONTRACTING METHOD" in c, columns)) 

    df = df.reindex(columns=new_columns)

    return df

In [11]:
setting = None
billing_class = 'professional'

df_mid = pd.DataFrame(df_phy)

df_mid = rearrange_columns(df_mid)
df_mid = df_mid.rename(columns={
    'CHG CD': 'local_code',
    'CHG CD DESC': 'description',
    'MCD CPT': 'hcpcs_cpt',
    'MOD': 'modifiers',
    'REV CD': 'rev_code',
    'MCD CPT/MS DRG': 'code',
    'Description': 'description',
    'CPT Code': 'hcpcs_cpt',
    'Mod': 'modifiers'
})

df_mid

,AETNA CONTRACTING METHOD,CIGNA CONTRACTING METHOD,EMBLEM CONTRACTING METHOD,EMPIRE CONTRACTING METHOD,UHC CONTRACTING METHOD,HEALTHFIRST CONTRACTING METHOD,MAGNACARE CONTRACTING METHOD,METROPLUS CONTRACTING METHOD,MULTIPLAN CONTRACTING METHOD,URN CONTRACTING METHOD,...,URN RATES,EMPIRE HEALTHPLUS RATES,EMBLEM HIP ESS 1&2 RATES,EMBLEM MCAID & HIP ESS 3&4 RATES,HEALTHFIRST ESS 1&2 RATES,HEALTHFIRST MCAID & ESS 3&4 RATES,FIDELIS ESS 1&2 RATES,FIDELIS MCAID & ESS 3&4 RATES,MIN PHY,MAX PHY
0,Conv Factor,Conv Factor,FEE SCHEDULE/PERCENT OF CHARGE,CONVERSION FACTOR,CONVERSION FACTOR,CONVERSION FACTOR,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,113.1,NaN,MEDICAID RATE,MEDICAID RATE,NaN,NaN,NaN,NaN,NaN,NaN
1,Conv Factor,Conv Factor,FEE SCHEDULE/PERCENT OF CHARGE,CONVERSION FACTOR,CONVERSION FACTOR,CONVERSION FACTOR,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,113.1,NaN,MEDICAID RATE,MEDICAID RATE,NaN,NaN,NaN,NaN,NaN,NaN
2,Conv Factor,Conv Factor,FEE SCHEDULE/PERCENT OF CHARGE,CONVERSION FACTOR,CONVERSION FACTOR,CONVERSION FACTOR,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,113.1,NaN,MEDICAID RATE,MEDICAID RATE,NaN,NaN,NaN,NaN,NaN,NaN
3,Conv Factor,Conv Factor,FEE SCHEDULE/PERCENT OF CHARGE,CONVERSION FACTOR,CONVERSION FACTOR,CONVERSION FACTOR,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,113.1,NaN,MEDICAID RATE,MEDICAID RATE,NaN,NaN,NaN,NaN,NaN,NaN
4,Conv Factor,Conv Factor,FEE SCHEDULE/PERCENT OF CHARGE,CONVERSION FACTOR,CONVERSION FACTOR,CONVERSION FACTOR,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,113.1,NaN,MEDICAID RATE,MEDICAID RATE,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12987,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,159.9,50.193,MEDICAID RATE,MEDICAID RATE,55.77,44.616,55.77,44.616,NaN,NaN
12988,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,159.9,50.193,MEDICAID RATE,MEDICAID RATE,55.77,44.616,55.77,44.616,NaN,NaN
12989,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,117.0,34.551,MEDICAID RATE,MEDICAID RATE,38.39,30.712,38.39,30.712,NaN,NaN
12990,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,885.3,NaN,MEDICAID RATE,MEDICAID RATE,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
money_idx = df_mid.columns.to_list().index("PRICE")
money_columns = df_mid.columns.to_list()[money_idx:]
money_columns

['PRICE',
 'SELF PAY',
 'AETNA COMMERCIAL NYC RATES',
 'AETNA PREMIER CARE NETWORK NYC RATES',
 'CIGNA RATES',
 'NYC EMBLEM GHI RATES',
 'NYC EMBLEM HIP RATES',
 'EMPIRE COMMERCIAL RATES',
 'EMPIRE BLUE ACCESS RATES',
 'EMPIRE CONNECTIONS RATES',
 'UHC COMMERCIAL RATES',
 'OXFORD FREEDOM RATES',
 'OXFORD LIBERTY RATES',
 'OXFORD METRO RATES',
 'UHC CORE RATES',
 'UHC CHARTER RATES',
 'HEALTHFIRST RATES',
 'MAGNACARE RATES',
 'MAGNACARE JIB RATES',
 'METROPLUS SMALL BUSINESS RATES',
 'METROPLUS CITY EMPLOYEE HEALTH RATES',
 'MULTIPLAN RATES',
 'URN RATES',
 'EMPIRE HEALTHPLUS RATES',
 'EMBLEM HIP ESS 1&2 RATES',
 'EMBLEM MCAID & HIP ESS 3&4 RATES',
 'HEALTHFIRST ESS 1&2 RATES',
 'HEALTHFIRST MCAID & ESS 3&4 RATES',
 'FIDELIS ESS 1&2 RATES',
 'FIDELIS MCAID & ESS 3&4 RATES',
 'MIN PHY',
 'MAX PHY']

In [13]:
remaining_columns = df_mid.columns.to_list()[:money_idx]
df_mid = pd.melt(df_mid, id_vars=remaining_columns, var_name='payer_name', value_name='standard_charge')
df_mid

,AETNA CONTRACTING METHOD,CIGNA CONTRACTING METHOD,EMBLEM CONTRACTING METHOD,EMPIRE CONTRACTING METHOD,UHC CONTRACTING METHOD,HEALTHFIRST CONTRACTING METHOD,MAGNACARE CONTRACTING METHOD,METROPLUS CONTRACTING METHOD,MULTIPLAN CONTRACTING METHOD,URN CONTRACTING METHOD,EMPIRE HEALTHPLUS CONTRACTING METHOD,HEALTHFIRST MCAID & ESS CONTRACTING METHODS,FIDELIS MCAID & ESS CONTRACTING METHODS,local_code,hcpcs_cpt,modifiers,description,payer_name,standard_charge
0,Conv Factor,Conv Factor,FEE SCHEDULE/PERCENT OF CHARGE,CONVERSION FACTOR,CONVERSION FACTOR,CONVERSION FACTOR,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,0100100,00100,,ANESTHESIA FOR SALIVARY GLANDS,PRICE,145
1,Conv Factor,Conv Factor,FEE SCHEDULE/PERCENT OF CHARGE,CONVERSION FACTOR,CONVERSION FACTOR,CONVERSION FACTOR,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,0100102,00102,,PLASTIC REPAIR OF CLEFT LIP,PRICE,145
2,Conv Factor,Conv Factor,FEE SCHEDULE/PERCENT OF CHARGE,CONVERSION FACTOR,CONVERSION FACTOR,CONVERSION FACTOR,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,0100103,00103,,BLEPHAROPLASTY,PRICE,145
3,Conv Factor,Conv Factor,FEE SCHEDULE/PERCENT OF CHARGE,CONVERSION FACTOR,CONVERSION FACTOR,CONVERSION FACTOR,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,0100104,00104,,ANESTH FOR ELECTROCONVULSIVE THERAPY,PRICE,145
4,Conv Factor,Conv Factor,FEE SCHEDULE/PERCENT OF CHARGE,CONVERSION FACTOR,CONVERSION FACTOR,CONVERSION FACTOR,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,0100120,00120,,ANESTHESIA FOR EAR SURGERY,PRICE,145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415739,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,9300452,G0452,26,MOLECULAR PATH PROC; INTRP AND RPT- TIER 1,MAX PHY,NaN
415740,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,9310452,G0452,26,MOLECULAR PATH PROC; INTRP AND RPT- TIER 2,MAX PHY,NaN
415741,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,9388387,88387,26,"MACROSCOPIC EXAM PREP,MOLEC STUDIES, EA TISS",MAX PHY,NaN
415742,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,J730842,J7308,,"AMINOLEVULINIC ACID HCL ADM,20%, 354MG - HAUP ...",MAX PHY,NaN


In [14]:
df_mid['standard_charge_percent'] = None

df_mid.loc[df_mid['payer_name'].str.endswith('PERCENT'), 'standard_charge_percent'] = df_mid[df_mid['payer_name'].str.endswith('PERCENT')]['standard_charge']
df_mid.loc[df_mid['payer_name'].str.endswith('PERCENT'), 'standard_charge'] = None

set(df_mid['standard_charge_percent'].to_list())

{None}

In [15]:
set(df_mid['modifiers'].to_list())

{' ',
 26,
 '26,50',
 '26,52',
 '26,52,50',
 '26,52,LT',
 '26,52,RT',
 '26,GA',
 '26,LT',
 '26,Q0',
 '26,RT',
 50,
 52,
 '52,26',
 58,
 59,
 '59,50',
 93,
 'GA',
 'GP',
 'GQ',
 'GT',
 'GY',
 'LT',
 'RT'}

In [16]:
def fix_modifiers(modifiers):
    if type(modifiers) == int:
        modifiers = str(modifiers)
        
    modifiers = modifiers.strip()
    if modifiers == '':
        return None

    modifiers = modifiers.replace(" ", "|")
    modifiers = modifiers.replace(",", "|")

    return modifiers

df_mid.loc[df_mid['modifiers'].notnull(), 'modifiers'] = df_mid[df_mid['modifiers'].notnull()]['modifiers'].apply(fix_modifiers)
set(df_mid['modifiers'].to_list())

{'26',
 '26|50',
 '26|52',
 '26|52|50',
 '26|52|LT',
 '26|52|RT',
 '26|GA',
 '26|LT',
 '26|Q0',
 '26|RT',
 '50',
 '52',
 '52|26',
 '58',
 '59',
 '59|50',
 '93',
 'GA',
 'GP',
 'GQ',
 'GT',
 'GY',
 'LT',
 None,
 'RT'}

In [17]:
def payer_name_to_rate_category(payer_name):
    if payer_name.startswith("MAX"):
        return 'max'
    elif payer_name.startswith("MIN"):
        return 'min'
    elif payer_name == "PRICE":
        return 'gross'
    elif payer_name == "SELF PAY":
        return 'cash'

    return 'negotiated'

df_mid['rate_category'] = df_mid['payer_name'].apply(payer_name_to_rate_category)
df_mid[['payer_name', 'rate_category']].value_counts()

payer_name                            rate_category
AETNA COMMERCIAL NYC RATES            negotiated       12992
AETNA PREMIER CARE NETWORK NYC RATES  negotiated       12992
CIGNA RATES                           negotiated       12992
EMBLEM HIP ESS 1&2 RATES              negotiated       12992
EMBLEM MCAID & HIP ESS 3&4 RATES      negotiated       12992
EMPIRE BLUE ACCESS RATES              negotiated       12992
EMPIRE COMMERCIAL RATES               negotiated       12992
EMPIRE CONNECTIONS RATES              negotiated       12992
EMPIRE HEALTHPLUS RATES               negotiated       12992
FIDELIS ESS 1&2 RATES                 negotiated       12992
FIDELIS MCAID & ESS 3&4 RATES         negotiated       12992
HEALTHFIRST ESS 1&2 RATES             negotiated       12992
HEALTHFIRST MCAID & ESS 3&4 RATES     negotiated       12992
HEALTHFIRST RATES                     negotiated       12992
MAGNACARE JIB RATES                   negotiated       12992
MAGNACARE RATES                  

In [18]:
df_mid['contracting_method'] = None
df_mid['additional_payer_specific_notes'] = None

def set_contracting_method(row):
    if row.get('rate_category') != "negotiated":
        return row
    
    method_by_payer = dict()
    
    for col_name in dict(row).keys():
        value = row[col_name]
        if type(value) != str:
            continue
        
        if 'CONTRACTING METHOD' in col_name:
            payer = col_name.replace("CONTRACTING METHOD", "")
            method_by_payer[payer] = value
    
    payer_name = row.get('payer_name')

    for payer in method_by_payer.keys():
        if not payer in payer_name:
            continue

        method = method_by_payer[payer]
        if method == 'FEE SCHEDULE':
            row['contracting_method'] = 'fee schedule'
        elif method == 'PERCENT OF CHARGE':
            row['contracting_method'] = 'percent of total billed charge'
        elif method == 'CASE RATE':
            row['contracting_method'] = 'case rate'
        elif method == 'PER DIEM':
            row['contracting_method'] = 'per diem'
        else:
            row['contracting_method'] = 'other'
            row['additional_payer_specific_notes'] = method

    return row

df_mid = df_mid.apply(set_contracting_method, axis=1)
df_mid[df_mid['rate_category'] == 'negotiated']

,AETNA CONTRACTING METHOD,CIGNA CONTRACTING METHOD,EMBLEM CONTRACTING METHOD,EMPIRE CONTRACTING METHOD,UHC CONTRACTING METHOD,HEALTHFIRST CONTRACTING METHOD,MAGNACARE CONTRACTING METHOD,METROPLUS CONTRACTING METHOD,MULTIPLAN CONTRACTING METHOD,URN CONTRACTING METHOD,...,local_code,hcpcs_cpt,modifiers,description,payer_name,standard_charge,standard_charge_percent,rate_category,contracting_method,additional_payer_specific_notes
25984,Conv Factor,Conv Factor,FEE SCHEDULE/PERCENT OF CHARGE,CONVERSION FACTOR,CONVERSION FACTOR,CONVERSION FACTOR,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,0100100,00100,None,ANESTHESIA FOR SALIVARY GLANDS,AETNA COMMERCIAL NYC RATES,119.33,None,negotiated,other,Conv Factor
25985,Conv Factor,Conv Factor,FEE SCHEDULE/PERCENT OF CHARGE,CONVERSION FACTOR,CONVERSION FACTOR,CONVERSION FACTOR,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,0100102,00102,None,PLASTIC REPAIR OF CLEFT LIP,AETNA COMMERCIAL NYC RATES,119.33,None,negotiated,other,Conv Factor
25986,Conv Factor,Conv Factor,FEE SCHEDULE/PERCENT OF CHARGE,CONVERSION FACTOR,CONVERSION FACTOR,CONVERSION FACTOR,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,0100103,00103,None,BLEPHAROPLASTY,AETNA COMMERCIAL NYC RATES,119.33,None,negotiated,other,Conv Factor
25987,Conv Factor,Conv Factor,FEE SCHEDULE/PERCENT OF CHARGE,CONVERSION FACTOR,CONVERSION FACTOR,CONVERSION FACTOR,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,0100104,00104,None,ANESTH FOR ELECTROCONVULSIVE THERAPY,AETNA COMMERCIAL NYC RATES,119.33,None,negotiated,other,Conv Factor
25988,Conv Factor,Conv Factor,FEE SCHEDULE/PERCENT OF CHARGE,CONVERSION FACTOR,CONVERSION FACTOR,CONVERSION FACTOR,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,0100120,00120,None,ANESTHESIA FOR EAR SURGERY,AETNA COMMERCIAL NYC RATES,119.33,None,negotiated,other,Conv Factor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389755,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,9300452,G0452,26,MOLECULAR PATH PROC; INTRP AND RPT- TIER 1,FIDELIS MCAID & ESS 3&4 RATES,44.616,None,negotiated,None,None
389756,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,9310452,G0452,26,MOLECULAR PATH PROC; INTRP AND RPT- TIER 2,FIDELIS MCAID & ESS 3&4 RATES,44.616,None,negotiated,None,None
389757,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,9388387,88387,26,"MACROSCOPIC EXAM PREP,MOLEC STUDIES, EA TISS",FIDELIS MCAID & ESS 3&4 RATES,30.712,None,negotiated,None,None
389758,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,J730842,J7308,None,"AMINOLEVULINIC ACID HCL ADM,20%, 354MG - HAUP ...",FIDELIS MCAID & ESS 3&4 RATES,NaN,None,negotiated,None,None


In [19]:
df_mid = df_mid[df_mid['standard_charge'].notnull() | df_mid['standard_charge_percent'].notnull()]
df_mid = df_mid[df_mid['standard_charge'] != 'MEDICAID RATE']
df_mid

,AETNA CONTRACTING METHOD,CIGNA CONTRACTING METHOD,EMBLEM CONTRACTING METHOD,EMPIRE CONTRACTING METHOD,UHC CONTRACTING METHOD,HEALTHFIRST CONTRACTING METHOD,MAGNACARE CONTRACTING METHOD,METROPLUS CONTRACTING METHOD,MULTIPLAN CONTRACTING METHOD,URN CONTRACTING METHOD,...,local_code,hcpcs_cpt,modifiers,description,payer_name,standard_charge,standard_charge_percent,rate_category,contracting_method,additional_payer_specific_notes
0,Conv Factor,Conv Factor,FEE SCHEDULE/PERCENT OF CHARGE,CONVERSION FACTOR,CONVERSION FACTOR,CONVERSION FACTOR,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,0100100,00100,None,ANESTHESIA FOR SALIVARY GLANDS,PRICE,145,None,gross,None,None
1,Conv Factor,Conv Factor,FEE SCHEDULE/PERCENT OF CHARGE,CONVERSION FACTOR,CONVERSION FACTOR,CONVERSION FACTOR,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,0100102,00102,None,PLASTIC REPAIR OF CLEFT LIP,PRICE,145,None,gross,None,None
2,Conv Factor,Conv Factor,FEE SCHEDULE/PERCENT OF CHARGE,CONVERSION FACTOR,CONVERSION FACTOR,CONVERSION FACTOR,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,0100103,00103,None,BLEPHAROPLASTY,PRICE,145,None,gross,None,None
3,Conv Factor,Conv Factor,FEE SCHEDULE/PERCENT OF CHARGE,CONVERSION FACTOR,CONVERSION FACTOR,CONVERSION FACTOR,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,0100104,00104,None,ANESTH FOR ELECTROCONVULSIVE THERAPY,PRICE,145,None,gross,None,None
4,Conv Factor,Conv Factor,FEE SCHEDULE/PERCENT OF CHARGE,CONVERSION FACTOR,CONVERSION FACTOR,CONVERSION FACTOR,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,0100120,00120,None,ANESTHESIA FOR EAR SURGERY,PRICE,145,None,gross,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389753,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,9288291,88291,None,DEB STRESS TEST INTERPRETATION,FIDELIS MCAID & ESS 3&4 RATES,29.44,None,negotiated,None,None
389754,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,9288292,88291,None,"CYTOGENETICS, MOLEC CYTOGENETICS, INTERP RPT",FIDELIS MCAID & ESS 3&4 RATES,29.44,None,negotiated,None,None
389755,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,9300452,G0452,26,MOLECULAR PATH PROC; INTRP AND RPT- TIER 1,FIDELIS MCAID & ESS 3&4 RATES,44.616,None,negotiated,None,None
389756,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,9310452,G0452,26,MOLECULAR PATH PROC; INTRP AND RPT- TIER 2,FIDELIS MCAID & ESS 3&4 RATES,44.616,None,negotiated,None,None


In [20]:
df_mid.loc[df_mid['standard_charge_percent'].notnull(), 'standard_charge_percent'] = df_mid[df_mid['standard_charge_percent'].notnull()]['standard_charge_percent'].apply(
    lambda percent: percent.replace("%", "").strip()
)

set(df_mid['standard_charge_percent'].to_list())

{None}

In [21]:
df_mid['standard_charge'] = df_mid['standard_charge'].apply(cleanup_dollar_value)
df_mid['standard_charge']

0         145.000
1         145.000
2         145.000
3         145.000
4         145.000
           ...   
389753     29.440
389754     29.440
389755     44.616
389756     44.616
389757     30.712
Name: standard_charge, Length: 326530, dtype: float64

In [22]:
def split_code(row):
    code = row.get('code')

    if code_is_cpt(code) or code_is_hcpcs(code):
        row['hcpcs_cpt'] = code
    elif code_is_ms_drg(code):
        row['ms_drg'] = code

    return row

if 'code' in df_mid.columns:
    df_mid.loc[df_mid['code'].isnull(), 'code'] = ''
    df_mid['code'] = df_mid['code'].astype(str)
    df_mid = df_mid.apply(split_code, axis=1)
    df_mid.loc[df_mid['code'] == '', 'code'] = None


In [23]:
import numpy as np

def fix_rev_code(rev_code):
    if rev_code is None or type(rev_code) == float and np.isnan(rev_code):
        return None

    # 260, 940
    if "," in rev_code:
        rev_code = rev_code.split(",")[0]

    # 450-459
    if "-" in rev_code:
        rev_code = rev_code.split("-")[0]

    rev_code = pad_rev_code_if_needed(rev_code)

    return rev_code

if not 'rev_code' in df_mid.columns:
    df_mid['rev_code'] = None

df_mid['rev_code'] = df_mid['rev_code'].apply(fix_rev_code)
df_mid.loc[df_mid['rev_code'] == 'Other IP', 'rev_code'] = None
set(df_mid['rev_code'].to_list())

{None}

In [24]:
df_mid['alt_hcpcs_cpt'] = None

# 96360-5, C8957
def fix_hcpcs_cpt(row):
    hcpcs_cpt = row.get('hcpcs_cpt')
    if not type(hcpcs_cpt) == str:
        return row

    if "," in hcpcs_cpt:
        codes = hcpcs_cpt.split(",")
        row['hcpcs_cpt'] = codes[0].strip()
        row['alt_hcpcs_cpt'] = codes[1].strip()

    hcpcs_cpt = row.get('hcpcs_cpt')
    if "-" in hcpcs_cpt:
        hcpcs_cpt, mod = hcpcs_cpt.split("-")
        mod = mod.strip()
        row['hcpcs_cpt'] = hcpcs_cpt
        if len(mod) == 2:
            if row.get('modifiers') is None:
                row['modifiers'] = mod
            else:
                row['modifiers'] = row['modifiers'] + "|" + mod

    return row

df_mid = df_mid.apply(fix_hcpcs_cpt, axis=1)
set(df_mid['alt_hcpcs_cpt'].to_list())

{None}

In [25]:
df_mid['hospital_id'] = ccn
df_mid['line_type'] = None
if not 'code' in df_mid.columns:
    df_mid['code'] = None
if not 'hcpcs_cpt' in df_mid.columns:
    df_mid['hcpcs_cpt'] = None
if not 'ms_drg' in df_mid.columns:
    df_mid['ms_drg'] = None
df_mid['apr_drg'] = None
df_mid['eapg'] = None
if not 'alt_hcpcs_cpt' in df_mid.columns:
    df_mid['alt_hcpcs_cpt'] = None
df_mid['thru'] = None
df_mid['apc'] = None
df_mid['icd'] = None
df_mid['ndc'] = None
df_mid['drug_hcpcs_multiplier'] = None
df_mid['drug_quantity'] = None
df_mid['drug_unit_of_measurement'] = None
df_mid['drug_type_of_measurement'] = None
df_mid['billing_class'] = billing_class
df_mid['setting'] = setting
df_mid['plan_name'] = None
df_mid['additional_generic_notes'] = None
if not 'additional_payer_specific_notes' in df_mid.columns:
    df_mid['additional_payer_specific_notes'] = None

df_mid

,AETNA CONTRACTING METHOD,CIGNA CONTRACTING METHOD,EMBLEM CONTRACTING METHOD,EMPIRE CONTRACTING METHOD,UHC CONTRACTING METHOD,HEALTHFIRST CONTRACTING METHOD,MAGNACARE CONTRACTING METHOD,METROPLUS CONTRACTING METHOD,MULTIPLAN CONTRACTING METHOD,URN CONTRACTING METHOD,...,icd,ndc,drug_hcpcs_multiplier,drug_quantity,drug_unit_of_measurement,drug_type_of_measurement,billing_class,setting,plan_name,additional_generic_notes
0,Conv Factor,Conv Factor,FEE SCHEDULE/PERCENT OF CHARGE,CONVERSION FACTOR,CONVERSION FACTOR,CONVERSION FACTOR,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,None,None,None,None,None,None,professional,None,None,None
1,Conv Factor,Conv Factor,FEE SCHEDULE/PERCENT OF CHARGE,CONVERSION FACTOR,CONVERSION FACTOR,CONVERSION FACTOR,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,None,None,None,None,None,None,professional,None,None,None
2,Conv Factor,Conv Factor,FEE SCHEDULE/PERCENT OF CHARGE,CONVERSION FACTOR,CONVERSION FACTOR,CONVERSION FACTOR,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,None,None,None,None,None,None,professional,None,None,None
3,Conv Factor,Conv Factor,FEE SCHEDULE/PERCENT OF CHARGE,CONVERSION FACTOR,CONVERSION FACTOR,CONVERSION FACTOR,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,None,None,None,None,None,None,professional,None,None,None
4,Conv Factor,Conv Factor,FEE SCHEDULE/PERCENT OF CHARGE,CONVERSION FACTOR,CONVERSION FACTOR,CONVERSION FACTOR,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,None,None,None,None,None,None,professional,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389753,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,None,None,None,None,None,None,professional,None,None,None
389754,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,None,None,None,None,None,None,professional,None,None,None
389755,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,None,None,None,None,None,None,professional,None,None,None
389756,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,FEE SCHEDULE,FEE SCHEDULE,FEE SCHEDULE,PERCENT OF CHARGE,FEE SCHEDULE,PERCENT OF CHARGE,PERCENT OF CHARGE,...,None,None,None,None,None,None,professional,None,None,None


In [26]:
df_out = pd.DataFrame(df_mid[TARGET_COLUMNS])
df_out

,hospital_id,line_type,description,rev_code,local_code,code,ms_drg,apr_drg,eapg,hcpcs_cpt,...,billing_class,setting,rate_category,payer_name,plan_name,standard_charge,standard_charge_percent,contracting_method,additional_generic_notes,additional_payer_specific_notes
0,330154,None,ANESTHESIA FOR SALIVARY GLANDS,None,0100100,None,None,None,None,00100,...,professional,None,gross,PRICE,None,145.000,None,None,None,None
1,330154,None,PLASTIC REPAIR OF CLEFT LIP,None,0100102,None,None,None,None,00102,...,professional,None,gross,PRICE,None,145.000,None,None,None,None
2,330154,None,BLEPHAROPLASTY,None,0100103,None,None,None,None,00103,...,professional,None,gross,PRICE,None,145.000,None,None,None,None
3,330154,None,ANESTH FOR ELECTROCONVULSIVE THERAPY,None,0100104,None,None,None,None,00104,...,professional,None,gross,PRICE,None,145.000,None,None,None,None
4,330154,None,ANESTHESIA FOR EAR SURGERY,None,0100120,None,None,None,None,00120,...,professional,None,gross,PRICE,None,145.000,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389753,330154,None,DEB STRESS TEST INTERPRETATION,None,9288291,None,None,None,None,88291,...,professional,None,negotiated,FIDELIS MCAID & ESS 3&4 RATES,None,29.440,None,None,None,None
389754,330154,None,"CYTOGENETICS, MOLEC CYTOGENETICS, INTERP RPT",None,9288292,None,None,None,None,88291,...,professional,None,negotiated,FIDELIS MCAID & ESS 3&4 RATES,None,29.440,None,None,None,None
389755,330154,None,MOLECULAR PATH PROC; INTRP AND RPT- TIER 1,None,9300452,None,None,None,None,G0452,...,professional,None,negotiated,FIDELIS MCAID & ESS 3&4 RATES,None,44.616,None,None,None,None
389756,330154,None,MOLECULAR PATH PROC; INTRP AND RPT- TIER 2,None,9310452,None,None,None,None,G0452,...,professional,None,negotiated,FIDELIS MCAID & ESS 3&4 RATES,None,44.616,None,None,None,None


In [27]:
df_out.to_csv('rate_tmp.csv', index=False)